# Bidirectional Recurrent Neural Network (Deep RNN)

In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import tensorflow as tf
from tensorflow.contrib.rnn.python.ops.core_rnn import static_bidirectional_rnn

----------------------------------------------------------------

## 1. Data (mnist)

In [2]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("D:/2_Edu/tmp/data/", one_hot=True)

Extracting D:/2_Edu/tmp/data/train-images-idx3-ubyte.gz
Extracting D:/2_Edu/tmp/data/train-labels-idx1-ubyte.gz
Extracting D:/2_Edu/tmp/data/t10k-images-idx3-ubyte.gz
Extracting D:/2_Edu/tmp/data/t10k-labels-idx1-ubyte.gz


----------------------------------------------------------------------------------

## 2. Bidirectional RNN

 - 시간 스텝 t에서의 출력값이 이전 시간 스텝 외에, 이후의 시간 스텝 입력값에도 영향을 받을 수 있다
 - 출력값은 두 RNN의 hidden state에 모두 의존하도록 계산


### 2-1. Hyper-parameter

In [3]:
training_iters  = 200000
batch_size      = 128
learning_rate   = 0.001
display_step    = 10

### 2-2. Layer parameter

 - consider every image row as a sequence of pixels
 - 28*28px >> 28 sequences of 28 steps

In [4]:
n_input   = 28
n_step    = 28
n_hidden  = 128
n_classes = 10

### 2-3. Input

In [5]:
x = tf.placeholder(tf.float32, [None, n_step, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])

weights = {'out': tf.Variable(tf.random_normal([2*n_hidden, n_classes]))}
biases  = {'out': tf.Variable(tf.random_normal([n_classes]))}

### 2-4. Modeling

In [6]:
def BiRNN(x, weights, biases):
    
    ## reshaping for RNN
    x = tf.transpose(x, [1,0,2])
    x = tf.reshape(x, [-1, n_input])
    x = tf.split(x, n_step, 0)

    ## LSTM
    ##-- fw: forward cell, bw: backward cell
    lstm_cell_fw = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)
    lstm_cell_bw = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias = 1.0)

    
    ## Cell Output
    outputs, _, _ = tf.contrib.rnn.static_bidirectional_rnn(lstm_cell_fw, lstm_cell_bw, x, dtype = tf.float32)

    return tf.matmul(outputs[-1], weights['out']) + biases['out']


model = BiRNN(x, weights, biases)
print(model)

Tensor("add:0", shape=(?, 10), dtype=float32)


### 2-5. Optimization

In [7]:
cost      = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = model, labels = y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

### 2-6. Test Accuracy

In [8]:
correct_model = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
accuracy      = tf.reduce_mean(tf.cast(correct_model, tf.float32))

### 2-7. tf.Session

In [9]:
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)

    step = 1
    while step * batch_size < training_iters:
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        batch_x = batch_x.reshape((batch_size, n_step, n_input))

        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y})

        if step % display_step == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={x: batch_x, y: batch_y})
            print("Iter " + str(step*batch_size) + \
                  ", Minibatch Loss= " + "{:.6f}".format(loss) +  \
                  ", Training Accuracy= " + "{:.5f}".format(acc))
        step += 1
    print("Optimization Finished!")
    print("==========================================================")

    ## 128 mnist test images
    test_len = 128
    test_data  = mnist.test.images[:test_len].reshape((-1, n_step, n_input))
    test_label = mnist.test.labels[:test_len]
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: test_data, y: test_label}))

Iter 1280, Minibatch Loss= 2.063506, Training Accuracy= 0.28906
Iter 2560, Minibatch Loss= 1.777920, Training Accuracy= 0.41406
Iter 3840, Minibatch Loss= 1.368550, Training Accuracy= 0.49219
Iter 5120, Minibatch Loss= 1.129501, Training Accuracy= 0.69531
Iter 6400, Minibatch Loss= 0.904674, Training Accuracy= 0.75781
Iter 7680, Minibatch Loss= 1.202785, Training Accuracy= 0.62500
Iter 8960, Minibatch Loss= 0.998449, Training Accuracy= 0.68750
Iter 10240, Minibatch Loss= 0.776043, Training Accuracy= 0.76562
Iter 11520, Minibatch Loss= 0.476647, Training Accuracy= 0.88281
Iter 12800, Minibatch Loss= 0.801440, Training Accuracy= 0.75000
Iter 14080, Minibatch Loss= 0.737565, Training Accuracy= 0.72656
Iter 15360, Minibatch Loss= 0.392288, Training Accuracy= 0.88281
Iter 16640, Minibatch Loss= 0.470018, Training Accuracy= 0.89844
Iter 17920, Minibatch Loss= 0.381047, Training Accuracy= 0.85938
Iter 19200, Minibatch Loss= 0.272885, Training Accuracy= 0.92188
Iter 20480, Minibatch Loss= 0.16

--------------------------------------------------------------------
*End of Code*